In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import os
import pathlib

model_path = '/content/audio_tflite_model.tflite'
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 170 129   1]
Input Type: <class 'numpy.float32'>
Output Shape: [1 4]
Output Type: <class 'numpy.float32'>


In [3]:
from scipy import signal
from scipy.io import wavfile 

rate, samples = wavfile.read ('/content/Recording_34.wav')

print (rate)
print (samples.shape)
freq, t, spec = signal.spectrogram(samples, rate)
print (freq.shape)
print (t.shape)

zero_padding = tf.zeros([22000] - tf.shape(samples), dtype=tf.float32)
waveform = tf.cast(samples, tf.float32)
equal_length = tf.concat([waveform, zero_padding], 0)
spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
spectrogram = tf.abs(spectrogram)
spec_new = tf.expand_dims(spectrogram, -1)

print (spec_new.shape)

spec2 = np.reshape(spec_new,(-1, spec_new.shape[0], spec_new.shape[1],1))

#print (spec1.shape)

interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], spec2)
interpreter.invoke()

predictionx = interpreter.get_tensor(output_details[0]['index'])

output = np.argmax(predictionx[0])

if output==0:
  print ('on')
elif output==1:
  print ('off')
elif output==2:
  print ('green')
elif output==3:
  print ('red')

16000
(11194,)
(129,)
(49,)
(170, 129, 1)
red
